# State Management for AI Agents - Simple Loan Approval Workflow

## Learning Objectives
- Understand why **state management** is critical for AI agents
- Learn how to preserve context across LLM calls
- Build a simple state machine for a multi-step workflow
- See the difference between stateless and stateful approaches

## The Problem: Stateless LLMs in Stateful Workflows

LLMs are **stateless** - each API call is independent. But real workflows need **state** to:
- Track progress through workflow phases (e.g., "we're in the approval stage")
- Keep data from previous steps (e.g., "we already validated the income")
- Make decisions based on accumulated context

## Example: Loan Approval Workflow

A 3-step workflow:
1. **VERIFY** - Verify applicant documents (deterministic)
2. **ASSESS** - Use LLM to assess risk (AI)
3. **DECIDE** - Make approval decision (rule-based)

Each step depends on data from the previous one!

## Setup

In [12]:
import os
from dataclasses import dataclass, field
from enum import Enum
from typing import Dict, List
from datetime import datetime
from openai import OpenAI

# Initialize OpenAI client with Vocareum endpoint
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✅ Environment ready!")

✅ Environment ready!


## Part 1: Define the State Machine

In [13]:
# Define workflow states
class LoanState(Enum):
    VERIFY = "verify"       # Step 1: Validate documents
    ASSESS = "assess"       # Step 2: Risk assessment with LLM
    DECIDE = "decide"       # Step 3: Make approval decision
    COMPLETED = "completed" # Done

# Sample loan applications
APPLICATIONS = [
    {"name": "Alice", "income": 85000, "credit_score": 750, "loan_amount": 50000, "employment_years": 5},
    {"name": "Bob", "income": 35000, "credit_score": 580, "loan_amount": 80000, "employment_years": 1},
]

print("✅ States and sample applications defined!")

✅ States and sample applications defined!


## Part 2: Define Context (State Preservation)

In [14]:
@dataclass
class LoanContext:
    """Application state maintained across workflow steps"""
    applicant_name: str
    income: float
    credit_score: int
    loan_amount: float
    employment_years: int
    
    # Results from each step
    documents_valid: bool = False
    risk_assessment: str = ""  # Will store LLM response
    final_decision: str = ""
    
    # Track workflow progress
    current_state: LoanState = LoanState.VERIFY
    state_history: List[str] = field(default_factory=list)
    
    def transition(self, new_state: LoanState):
        """Record state transition"""
        self.state_history.append(f"{self.current_state.value} → {new_state.value}")
        self.current_state = new_state
        print(f"🔄 {self.state_history[-1]}")

print("✅ Context class defined!")

✅ Context class defined!


## Part 3: Build the Workflow

In [15]:
class LoanProcessor:
    """Stateful loan approval workflow"""
    
    def __init__(self, app: dict):
        """Initialize with applicant data"""
        self.context = LoanContext(
            applicant_name=app["name"],
            income=app["income"],
            credit_score=app["credit_score"],
            loan_amount=app["loan_amount"],
            employment_years=app["employment_years"]
        )
    
    def verify_documents(self):
        """Step 1: Verify applicant documents (deterministic)"""
        print(f"\n1️⃣ VERIFY: Checking {self.context.applicant_name}'s documents...")
        
        # Simple validation rules
        self.context.documents_valid = (
            self.context.income > 0 and 
            self.context.credit_score >= 300 and
            self.context.employment_years >= 1
        )
        
        if self.context.documents_valid:
            print(f"   ✅ Documents valid")
            print(f"      • Income: ${self.context.income:,.0f}")
            print(f"      • Credit Score: {self.context.credit_score}")
            print(f"      • Employment: {self.context.employment_years} years")
        else:
            print(f"   ❌ Documents invalid - missing required fields")
            return False
        
        self.context.transition(LoanState.ASSESS)
        return True
    
    def assess_risk(self):
        """Step 2: Use LLM to assess risk (AI-assisted)"""
        print(f"\n2️⃣ ASSESS: Getting AI risk assessment...")
        
        # Use verified data from Step 1
        prompt = f"""Assess the risk of approving this loan application. Be concise (2-3 sentences).

Applicant: {self.context.applicant_name}
Income: ${self.context.income:,.0f}
Credit Score: {self.context.credit_score}
Loan Amount: ${self.context.loan_amount:,.0f}
Employment: {self.context.employment_years} years
Debt-to-Income Ratio: {(self.context.loan_amount / self.context.income):.1%}

Provide: Risk level (LOW/MEDIUM/HIGH) and key concerns."""
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            max_tokens=100,
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Store LLM assessment in context for Step 3
        self.context.risk_assessment = response.choices[0].message.content
        print(f"   Assessment: {self.context.risk_assessment}")
        
        self.context.transition(LoanState.DECIDE)
    
    def make_decision(self):
        """Step 3: Make final decision (rule-based, using context from Steps 1 & 2)"""
        print(f"\n3️⃣ DECIDE: Making final decision...")
        
        # Use data from Step 1 and Step 2
        debt_ratio = self.context.loan_amount / self.context.income
        has_high_risk = "HIGH" in self.context.risk_assessment
        
        # Decision rules
        if debt_ratio > 1.0 or (self.context.credit_score < 600 and has_high_risk):
            self.context.final_decision = "❌ DENIED"
            reason = "High debt-to-income ratio or poor credit with high risk assessment"
        elif self.context.credit_score >= 700:
            self.context.final_decision = "✅ APPROVED"
            reason = "Strong credit score and acceptable risk profile"
        else:
            self.context.final_decision = "⏳ REVIEW"
            reason = "Borderline case - requires manual review"
        
        print(f"   Decision: {self.context.final_decision}")
        print(f"   Reason: {reason}")
        
        self.context.transition(LoanState.COMPLETED)
    
    def process(self):
        """Run the complete workflow"""
        print(f"\n{'='*50}")
        print(f"💰 LOAN APPROVAL: {self.context.applicant_name}")
        print(f"{'='*50}")
        
        # Run all steps
        if self.verify_documents():
            self.assess_risk()
            self.make_decision()
        else:
            print("❌ Application rejected at verification stage")
        
        print(f"\n📋 Workflow State History:")
        for step in self.context.state_history:
            print(f"   • {step}")
        
        return self.context

print("✅ LoanProcessor class defined!")

✅ LoanProcessor class defined!


## Part 4: Run the Workflow

In [16]:
# Process all applications
print("🏦 PROCESSING LOAN APPLICATIONS")
print("="*50)

for app in APPLICATIONS:
    processor = LoanProcessor(app)
    processor.process()

print(f"\n✅ All applications processed!")

🏦 PROCESSING LOAN APPLICATIONS

💰 LOAN APPROVAL: Alice

1️⃣ VERIFY: Checking Alice's documents...
   ✅ Documents valid
      • Income: $85,000
      • Credit Score: 750
      • Employment: 5 years
🔄 verify → assess

2️⃣ ASSESS: Getting AI risk assessment...
   Assessment: Risk Level: HIGH  
Key Concerns: Alice's debt-to-income ratio of 58.8% is significantly above the recommended threshold of 36%, indicating a high level of existing debt relative to her income, which could impair her ability to manage additional loan payments effectively.
🔄 assess → decide

3️⃣ DECIDE: Making final decision...
   Decision: ✅ APPROVED
   Reason: Strong credit score and acceptable risk profile
🔄 decide → completed

📋 Workflow State History:
   • verify → assess
   • assess → decide
   • decide → completed

💰 LOAN APPROVAL: Bob

1️⃣ VERIFY: Checking Bob's documents...
   ✅ Documents valid
      • Income: $35,000
      • Credit Score: 580
      • Employment: 1 years
🔄 verify → assess

2️⃣ ASSESS: Getting A

## Part 5: Comparison - What's Wrong Without State Management?

In [17]:
print("\n" + "="*60)
print("❌ PROBLEM: Without State Management")
print("="*60)

def broken_loan_approval(app):
    """WRONG approach: No state management"""
    print(f"\n🔴 Processing {app['name']} without state management...")
    
    # Step 1: Verify
    print(f"  Step 1: Verify documents")
    verified = app["income"] > 0 and app["credit_score"] >= 300
    print(f"  ✓ Verified: {verified}")
    
    # Step 2: Assess - but we lost the verification result!
    print(f"  Step 2: Assess risk")
    prompt = f"Risk for {app['name']} with ${app['income']:,.0f} income and {app['credit_score']} credit score?"
    print(f"  ⚠️  Had to recreate all data in the prompt")
    
    # Step 3: Decide - but we lost both previous results!
    print(f"  Step 3: Make decision")
    print(f"  ⚠️  Had to recalculate debt ratio")
    print(f"  ⚠️  Lost the risk assessment from Step 2")
    print(f"  ⚠️  No audit trail of what happened")
    print(f"  ⚠️  Can't easily resume if something fails")

broken_loan_approval(APPLICATIONS[0])

print("\n" + "="*60)
print("✅ SOLUTION: With State Management")
print("="*60)
print("✓ Context preserved across all steps")
print("✓ Each step uses verified data from previous steps")
print("✓ Complete audit trail of decisions")
print("✓ Easy to debug and resume workflows")
print("✓ Scales to complex multi-step workflows")


❌ PROBLEM: Without State Management

🔴 Processing Alice without state management...
  Step 1: Verify documents
  ✓ Verified: True
  Step 2: Assess risk
  ⚠️  Had to recreate all data in the prompt
  Step 3: Make decision
  ⚠️  Had to recalculate debt ratio
  ⚠️  Lost the risk assessment from Step 2
  ⚠️  No audit trail of what happened
  ⚠️  Can't easily resume if something fails

✅ SOLUTION: With State Management
✓ Context preserved across all steps
✓ Each step uses verified data from previous steps
✓ Complete audit trail of decisions
✓ Easy to debug and resume workflows
✓ Scales to complex multi-step workflows


## Summary

### Key Concepts Demonstrated

**1. State Machine**: Clear workflow stages (VERIFY → ASSESS → DECIDE → COMPLETED)

**2. Context Preservation**: `LoanContext` holds all data across steps
- Verified facts from Step 1 available in Step 3
- LLM response from Step 2 available in Step 3
- No data loss or recalculation needed

**3. LLM Integration**: AI is used at specific points (Step 2) within a deterministic workflow
- Not replacing the entire workflow with LLM
- AI augments human-defined business logic
- Results can be validated against rules

**4. Audit Trail**: Complete history of workflow execution
- Every state transition is recorded
- Decisions are traceable to specific steps
- Regulatory compliance requirement

### Why This Pattern Matters

- **Reliability**: Deterministic workflow with clear states
- **Observability**: Full audit trail for debugging
- **Compliance**: Meets regulatory requirements for decision documentation
- **Scalability**: Works for simple 3-step workflows or complex 20-step workflows
- **LLM Limits**: Overcomes statelessness of LLMs

### Real-World Applications

This pattern is used in production for:
- ✅ Loan/credit approval workflows
- ✅ Compliance and KYC processes
- ✅ Insurance claim processing
- ✅ Customer onboarding
- ✅ Order fulfillment
- ✅ Any multi-step decision workflow